# 📈 Stock Trading Analysis Tool (Updated)
## Analyze Your Buy & Sell Decisions Against Moving Averages

This notebook will help you:
1. Download 5 years of historical data for all your stocks
2. Calculate 50-DMA and 200-DMA
3. Analyze each buy transaction (conditions at entry)
4. Analyze each sell transaction (conditions at exit)
5. Determine if you made the right calls
6. Calculate opportunity cost/gain

## Step 1: Install Required Libraries

In [1]:
!pip install yfinance openpyxl -q
print("✓ Libraries installed successfully!")

✓ Libraries installed successfully!


## Step 2: Upload Your Tax P&L Excel File

Click on the file upload button below and select your tax P&L Excel file

In [10]:
from google.colab import files
import pandas as pd

print("=" * 80)
print("TAX P&L FILE UPLOAD")
print("=" * 80)

# Ask user how many files they want to upload
while True:
    try:
        num_files = int(input("\nHow many Tax P&L files do you want to upload? (Enter a number): "))
        if num_files > 0:
            break
        else:
            print("Please enter a positive number.")
    except ValueError:
        print("Invalid input. Please enter a number.")

print(f"\n✓ You will upload {num_files} file(s)")
print("─" * 80)

# Upload files one by one
all_dataframes = []
uploaded_filenames = []

for i in range(num_files):
    print(f"\n📁 Upload File {i+1} of {num_files}:")
    uploaded = files.upload()

    # Get the filename
    filename = list(uploaded.keys())[0]
    uploaded_filenames.append(filename)
    print(f"✓ File '{filename}' uploaded successfully!")

    # Read the Excel file (header is at row 19)
    temp_df = pd.read_excel(filename, skiprows=19)

    # Remove any rows that might be headers or empty
    temp_df = temp_df[temp_df['Symbol'].notna()].copy()
    temp_df = temp_df[temp_df['Symbol'] != 'Symbol'].copy()  # Remove if header row was included

    # Rename 'Profit' column to 'Actualised Profit'
    temp_df = temp_df.rename(columns={'Profit': 'Actualised Profit'})

    # Convert dates (with error handling)
    temp_df['Entry Date'] = pd.to_datetime(temp_df['Entry Date'], errors='coerce')
    temp_df['Exit Date'] = pd.to_datetime(temp_df['Exit Date'], errors='coerce')

    # Remove rows with invalid dates
    temp_df = temp_df[temp_df['Entry Date'].notna() & temp_df['Exit Date'].notna()].copy()

    # Convert numeric columns
    numeric_cols = ['Quantity', 'Buy Value', 'Sell Value', 'Actualised Profit']
    for col in numeric_cols:
        temp_df[col] = pd.to_numeric(temp_df[col], errors='coerce')

    # Remove any rows with NaN in critical columns
    temp_df = temp_df[temp_df['Quantity'].notna() & temp_df['Buy Value'].notna() & temp_df['Sell Value'].notna()].copy()

    all_dataframes.append(temp_df)
    print(f"   - Contains {len(temp_df)} trades")

# Combine all dataframes
print("\n" + "─" * 80)
print("COMBINING FILES...")
print("─" * 80)

df = pd.concat(all_dataframes, ignore_index=True)

print(f"\n✓ All {num_files} file(s) combined successfully!")
print(f"\nFiles uploaded:")
for idx, fname in enumerate(uploaded_filenames, 1):
    print(f"   {idx}. {fname}")

# Remove duplicates if any (based on key columns)
#initial_count = len(df)
#df = df.drop_duplicates(subset=['Symbol', 'Entry Date', 'Exit Date', 'Quantity', 'Buy Value', 'Sell Value'])
#duplicates_removed = initial_count - len(df)

#if duplicates_removed > 0:
 #   print(f"\n⚠️  Removed {duplicates_removed} duplicate trade(s)")

# Reset index
df = df.reset_index(drop=True)

print("\n" + "=" * 80)
print("COMBINED DATA SUMMARY")
print("=" * 80)
print(f"\nTotal trades: {len(df)}")
print(f"Profitable trades: {len(df[df['Actualised Profit'] > 0])}")
print(f"Loss-making trades: {len(df[df['Actualised Profit'] < 0])}")
print(f"Unique stocks: {df['Symbol'].nunique()}")

# Show date range
print(f"\nDate range: {df['Entry Date'].min().strftime('%Y-%m-%d')} to {df['Exit Date'].max().strftime('%Y-%m-%d')}")

# Show stock list
print(f"\nTop 10 most traded stocks:")
stock_counts = df['Symbol'].value_counts()
for stock, count in stock_counts.head(10).items():
    print(f"   {stock}: {count} trades")
if len(stock_counts) > 10:
    print(f"   ... and {len(stock_counts) - 10} more stocks")

print("\n" + "─" * 80)
print("First few rows of combined data:")
print("─" * 80)
df[['Symbol', 'Entry Date', 'Exit Date', 'Quantity', 'Buy Value', 'Sell Value', 'Actualised Profit']].head(10)


TAX P&L FILE UPLOAD

How many Tax P&L files do you want to upload? (Enter a number): 1

✓ You will upload 1 file(s)
────────────────────────────────────────────────────────────────────────────────

📁 Upload File 1 of 1:


Saving taxpnl-SSW336-2025_2026-Q1-Q3 2.xlsx to taxpnl-SSW336-2025_2026-Q1-Q3 2 (1).xlsx
✓ File 'taxpnl-SSW336-2025_2026-Q1-Q3 2 (1).xlsx' uploaded successfully!
   - Contains 754 trades

────────────────────────────────────────────────────────────────────────────────
COMBINING FILES...
────────────────────────────────────────────────────────────────────────────────

✓ All 1 file(s) combined successfully!

Files uploaded:
   1. taxpnl-SSW336-2025_2026-Q1-Q3 2 (1).xlsx

COMBINED DATA SUMMARY

Total trades: 754
Profitable trades: 470
Loss-making trades: 284
Unique stocks: 75

Date range: 2024-11-14 to 2025-10-24

Top 10 most traded stocks:
   KIMS: 49 trades
   TARIL: 46 trades
   INDHOTEL: 33 trades
   GOLDETF: 31 trades
   E2E: 29 trades
   NETWEB: 28 trades
   JAGSNPHARM: 28 trades
   JASH: 28 trades
   HCG: 25 trades
   ELECON: 24 trades
   ... and 65 more stocks

────────────────────────────────────────────────────────────────────────────────
First few rows of combined data:
────────

,Symbol,Entry Date,Exit Date,Quantity,Buy Value,Sell Value,Actualised Profit
0,BAJFINANCE,2025-04-30,2025-04-30,1,8685.05,8644.50,-40.55
1,HCG,2025-05-27,2025-05-27,5,2641.25,2760.25,119.00
2,HCG,2025-05-27,2025-05-27,15,7923.75,8280.00,356.25
3,ETERNAL,2025-05-30,2025-05-30,1,235.00,236.07,1.07
4,CDSL,2025-06-02,2025-06-02,20,31416.00,32448.00,1032.00
5,ABB,2025-08-04,2025-08-04,5,25885.00,25445.00,-440.00
6,TARIL,2025-09-30,2025-09-30,10,4796.50,4818.50,22.00
7,ZOMATO,2025-01-21,2025-04-16,10,2146.00,2210.20,64.20
8,ZOMATO,2025-03-10,2025-04-16,7,1498.63,1547.14,48.51
9,ZOMATO,2025-03-10,2025-04-16,11,2354.99,2431.22,76.23


## Step 3: Import Required Functions

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


## Step 4: Define Analysis Functions

In [4]:
def download_stock_data(symbol, period='5y'):
    """
    Download stock data from Yahoo Finance
    Args:
        symbol: Stock symbol (without .NS)
        period: Time period for historical data
    Returns:
        DataFrame with historical data
    """
    try:
        # Remove -E suffix if present (for ETFs traded on BSE)
        clean_symbol = symbol.split('-')[0]

        # Add .NS for NSE stocks
        ticker = f"{clean_symbol}.NS"
        stock = yf.Ticker(ticker)
        data = stock.history(period=period)

        if data.empty:
            print(f"Warning: No data found for {symbol}")
            return None

        return data
    except Exception as e:
        print(f"Error downloading {symbol}: {str(e)}")
        return None

def calculate_moving_averages(data):
    """Calculate 50-day and 200-day moving averages"""
    if data is None or data.empty:
        return None

    data['50_DMA'] = data['Close'].rolling(window=50).mean()
    data['200_DMA'] = data['Close'].rolling(window=200).mean()

    return data

def get_price_vs_dma(symbol, date, stock_data):
    """Get price difference vs DMAs on a specific date"""
    target_date = pd.to_datetime(date).date()

    if symbol not in stock_data or stock_data[symbol] is None:
        return "No data available"

    df = stock_data[symbol]
    available_dates = df.index.date
    closest_date = min(available_dates, key=lambda x: abs((x - target_date).days))

    if abs((closest_date - target_date).days) > 5:
        return "Date too far from available data"

    try:
        day_data = df[df.index.date == closest_date].iloc[0]
        price = day_data['Close']
        dma_50 = day_data['50_DMA']
        dma_200 = day_data['200_DMA']

        if pd.isna(dma_50) or pd.isna(dma_200):
            return "DMA not available (insufficient history)"

        diff_50 = ((price - dma_50) / dma_50) * 100
        diff_200 = ((price - dma_200) / dma_200) * 100

        return f"50DMA: {diff_50:+.2f}%, 200DMA: {diff_200:+.2f}%"

    except Exception as e:
        return f"Error: {str(e)}"

def get_current_price_and_analysis(row, stock_data):
    """Get current price and calculate profit/loss vs sell date"""
    symbol = row['Symbol']
    sell_price = row['Sell Value'] / row['Quantity']  # Calculate per-unit sell price
    quantity = row['Quantity']

    if symbol not in stock_data or stock_data[symbol] is None:
        return None, None, "No data", None

    df = stock_data[symbol]

    try:
        current_price = df['Close'].iloc[-1]
        price_diff_pct = ((current_price - sell_price) / sell_price) * 100
        verdict = "Good call" if price_diff_pct < 0 else "Wrong early sell"
        profit_loss = (current_price - sell_price) * quantity

        return current_price, price_diff_pct, verdict, profit_loss

    except Exception as e:
        print(f"Error analyzing {symbol}: {str(e)}")
        return None, None, "Error", None

print("✓ Functions defined successfully!")

✓ Functions defined successfully!


## Step 5: Download Historical Data for All Stocks

This will take a few minutes depending on the number of unique stocks...

In [5]:
# Get unique stock symbols
unique_stocks = df['Symbol'].unique()

print(f"Downloading data for {len(unique_stocks)} unique stocks...\n")

# Download and process data for each stock
stock_data = {}
for idx, symbol in enumerate(unique_stocks, 1):
    print(f"[{idx}/{len(unique_stocks)}] Downloading {symbol}...", end=' ')
    data = download_stock_data(symbol)

    if data is not None:
        data = calculate_moving_averages(data)
        stock_data[symbol] = data
        print(f"✓ ({len(data)} days)")
    else:
        print("✗ (Failed)")
        stock_data[symbol] = None

print(f"\n✓ Downloaded data for {sum(1 for v in stock_data.values() if v is not None)}/{len(unique_stocks)} stocks")


[1/75] Downloading BAJFINANCE... ✓ (1237 days)
[2/75] Downloading HCG... ✓ (1237 days)
[3/75] Downloading ETERNAL... ✓ (1032 days)
[4/75] Downloading CDSL... ✓ (1237 days)
[5/75] Downloading ABB... ✓ (1237 days)
[6/75] Downloading TARIL... ✓ (1237 days)
[7/75] Downloading ZOMATO... 

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ZOMATO.NS"}}}
ERROR:yfinance:$ZOMATO.NS: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


✗ (Failed)
[8/75] Downloading KSCL... ✓ (1237 days)
[9/75] Downloading SARDAEN... ✓ (1237 days)
[10/75] Downloading KRN... ✓ (264 days)
[11/75] Downloading MEDANTA... ✓ (725 days)
[12/75] Downloading CHAMBLFERT... ✓ (1237 days)
[13/75] Downloading INDIASHLTR... ✓ (456 days)
[14/75] Downloading AVANTIFEED... ✓ (1237 days)
[15/75] Downloading AAVAS... ✓ (1237 days)
[16/75] Downloading AARTIPHARM... ✓ (675 days)
[17/75] Downloading CAMS... ✓ (1237 days)
[18/75] Downloading INDHOTEL... ✓ (1237 days)
[19/75] Downloading ASTRAZEN... ✓ (1237 days)
[20/75] Downloading BAJAJFINSV... ✓ (1237 days)
[21/75] Downloading CHOLAFIN... ✓ (1237 days)
[22/75] Downloading ASALCBR... ✓ (1237 days)
[23/75] Downloading JLHL... ✓ (518 days)
[24/75] Downloading SHAILY... ✓ (880 days)
[25/75] Downloading BAJAJHLDNG... ✓ (1237 days)
[26/75] Downloading JASH... ✓ (1237 days)
[27/75] Downloading HAL... ✓ (1237 days)
[28/75] Downloading JAGSNPHARM... ✓ (1237 days)
[29/75] Downloading JUBLFOOD... ✓ (1237 days)
[30/7

## Step 6: Analyze All Trades

Analyzing buy conditions, sell conditions, and verdicts...

In [6]:
print("\nAnalyzing trades...\n")
print("─" * 80)

# Create result dataframe
result_df = df[['Symbol', 'Entry Date', 'Exit Date', 'Quantity', 'Buy Value', 'Sell Value', 'Actualised Profit']].copy()

# Add Buy Conditions (conditions at entry date)
print("Analyzing buy conditions...")
result_df['Buy Conditions'] = result_df.apply(
    lambda row: get_price_vs_dma(row['Symbol'], row['Entry Date'], stock_data),
    axis=1
)

# Add Sell Conditions (conditions at exit date)
print("Analyzing sell conditions...")
result_df['Sell Conditions'] = result_df.apply(
    lambda row: get_price_vs_dma(row['Symbol'], row['Exit Date'], stock_data),
    axis=1
)

# Add Buy Verdict based on Actualised Profit
result_df['Buy Verdict'] = result_df['Actualised Profit'].apply(
    lambda x: "Good call" if x > 0 else "Bad call" if x < 0 else "Break-even"
)

# Add Current Analysis (for sell decisions)
print("Analyzing current prices vs sell prices...")
analysis_results = result_df.apply(
    lambda row: get_current_price_and_analysis(row, stock_data),
    axis=1
)

result_df['Current Price'] = analysis_results.apply(lambda x: x[0])
result_df['Price Change Since Sell'] = analysis_results.apply(lambda x: f"{x[1]:.2f}%" if x[1] is not None else "N/A")
result_df['Sell Verdict'] = analysis_results.apply(lambda x: x[2])
result_df['Opportunity Cost/Gain'] = analysis_results.apply(lambda x: f"{x[3]:,.2f}" if x[3] is not None else "N/A")

# Format Actualised Profit for display
result_df['Actualised Profit (Formatted)'] = result_df['Actualised Profit'].apply(lambda x: f"{x:,.2f}")

# Reorder columns for better readability
final_columns = [
    'Symbol',
    'Entry Date',
    'Buy Value',
    'Buy Conditions',
    'Exit Date',
    'Sell Value',
    'Sell Conditions',
    'Quantity',
    'Actualised Profit (Formatted)',
    'Buy Verdict',
    'Current Price',
    'Price Change Since Sell',
    'Sell Verdict',
    'Opportunity Cost/Gain'
]

result_df_display = result_df[final_columns].copy()

print("\n✓ Analysis complete!")
print("─" * 80)


Analyzing trades...

────────────────────────────────────────────────────────────────────────────────
Analyzing buy conditions...
Analyzing sell conditions...
Analyzing current prices vs sell prices...

✓ Analysis complete!
────────────────────────────────────────────────────────────────────────────────


## Step 7: View Results

In [7]:
print("\n" + "=" * 80)
print("ANALYSIS RESULTS")
print("=" * 80)

# Show summary statistics
good_buys = len(result_df[result_df['Buy Verdict'] == 'Good call'])
bad_buys = len(result_df[result_df['Buy Verdict'] == 'Bad call'])
good_sells = len(result_df[result_df['Sell Verdict'] == 'Good call'])
wrong_sells = len(result_df[result_df['Sell Verdict'] == 'Wrong early sell'])

print(f"\n📊 SUMMARY:")
print(f"   Buy Decisions: {good_buys} profitable, {bad_buys} loss-making")
print(f"   Sell Timing: {good_sells} good calls (price fell), {wrong_sells} early sells (price rose)")
print(f"   Success Rate: {(good_buys/len(result_df)*100):.1f}% profitable trades")

print("\n" + "─" * 80)
print("Detailed Results:")
print("─" * 80)

result_df_display


ANALYSIS RESULTS

📊 SUMMARY:
   Buy Decisions: 451 profitable, 268 loss-making
   Sell Timing: 308 good calls (price fell), 401 early sells (price rose)
   Success Rate: 62.7% profitable trades

────────────────────────────────────────────────────────────────────────────────
Detailed Results:
────────────────────────────────────────────────────────────────────────────────


,Symbol,Entry Date,Buy Value,Buy Conditions,Exit Date,Sell Value,Sell Conditions,Quantity,Actualised Profit (Formatted),Buy Verdict,Current Price,Price Change Since Sell,Sell Verdict,Opportunity Cost/Gain
0,BAJFINANCE,2025-04-30,8685.05,"50DMA: -0.93%, 200DMA: +14.93%",2025-04-30,8644.50,"50DMA: -0.93%, 200DMA: +14.93%",1,-40.55,Bad call,1089.750000,-87.39%,Good call,"-7,554.75"
1,HCG,2025-05-27,2641.25,"50DMA: -3.61%, 200DMA: +10.36%",2025-05-27,2760.25,"50DMA: -3.61%, 200DMA: +10.36%",5,119.00,Good call,759.099976,37.51%,Wrong early sell,"1,035.25"
2,HCG,2025-05-27,7923.75,"50DMA: -3.61%, 200DMA: +10.36%",2025-05-27,8280.00,"50DMA: -3.61%, 200DMA: +10.36%",15,356.25,Good call,759.099976,37.52%,Wrong early sell,"3,106.50"
3,ETERNAL,2025-05-30,235.00,"50DMA: +5.68%, 200DMA: -4.18%",2025-05-30,236.07,"50DMA: +5.68%, 200DMA: -4.18%",1,1.07,Good call,326.600006,38.35%,Wrong early sell,90.53
4,CDSL,2025-06-02,31416.00,"50DMA: +28.90%, 200DMA: +16.72%",2025-06-02,32448.00,"50DMA: +28.90%, 200DMA: +16.72%",20,"1,032.00",Good call,1590.199951,-1.98%,Good call,-644.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,GOLDETF,2025-10-13,3367.56,"50DMA: +14.45%, 200DMA: +29.74%",2025-10-24,3306.52,"50DMA: +8.18%, 200DMA: +24.77%",28,-61.04,Bad call,118.059998,-0.03%,Good call,-0.84
715,GOLDETF,2025-10-13,44785.08,"50DMA: +14.45%, 200DMA: +29.74%",2025-10-24,43929.48,"50DMA: +8.18%, 200DMA: +24.77%",372,-855.60,Bad call,118.059998,-0.03%,Good call,-11.16
716,SILVERBEES,2025-10-23,1361.10,"50DMA: +9.75%, 200DMA: +35.58%",2025-10-23,1431.00,"50DMA: +9.75%, 200DMA: +35.58%",10,69.90,Good call,138.750000,-3.04%,Good call,-43.50
717,GOLDETF,2025-10-23,115.57,"50DMA: +10.01%, 200DMA: +26.68%",2025-10-24,118.09,"50DMA: +8.18%, 200DMA: +24.77%",1,2.52,Good call,118.059998,-0.03%,Good call,-0.03


## Step 8: Detailed Summary Analytics

Breaking down your trading performance by financial year...

In [9]:
from datetime import datetime

def get_financial_year(date):
    date = pd.to_datetime(date)
    if date.month >= 4:
        return f"FY {date.year}-{str(date.year + 1)[-2:]}"
    else:
        return f"FY {date.year - 1}-{str(date.year)[-2:]}"

result_df['Entry FY'] = result_df['Entry Date'].apply(get_financial_year)
result_df['Exit FY'] = result_df['Exit Date'].apply(get_financial_year)

financial_years = sorted(result_df['Exit FY'].unique())

print("\n" + "=" * 80)
print("TRADING PERFORMANCE ANALYSIS - BY FINANCIAL YEAR")
print("=" * 80)

for fy in financial_years:
    fy_data = result_df[result_df['Exit FY'] == fy]

    print(f"\n{'─' * 80}")
    print(f"📅 {fy} ({len(fy_data)} trades)")
    print(f"{'─' * 80}")

    print("\n📊 Buy Decision Performance:")
    buy_verdict_counts = fy_data['Buy Verdict'].value_counts()
    for verdict, count in buy_verdict_counts.items():
        percentage = (count / len(fy_data)) * 100
        print(f"   {verdict}: {count} trades ({percentage:.1f}%)")

    fy_actual_profit = fy_data['Actualised Profit'].sum()

    print(f"\n💰 Total Actualised Profit/Loss:")
    print(f"   ₹{fy_actual_profit:,.2f}")
    if fy_actual_profit > 0:
        print(f"   Note: Net profit from all trades closed in {fy}")
    else:
        print(f"   Note: Net loss from all trades closed in {fy}")

    print("\n📊 Sell Timing Analysis:")
    sell_verdict_counts = fy_data['Sell Verdict'].value_counts()
    for verdict, count in sell_verdict_counts.items():
        if verdict not in ['No data', 'Error']:
            percentage = (count / len(fy_data)) * 100
            print(f"   {verdict}: {count} trades ({percentage:.1f}%)")

    fy_data_copy = fy_data.copy()
    fy_data_copy['Opp_numeric'] = fy_data_copy['Opportunity Cost/Gain'].apply(
        lambda x: float(x.replace(',', '')) if isinstance(x, str) and x != 'N/A' else 0
    )
    fy_opportunity = fy_data_copy['Opp_numeric'].sum()

    print(f"\n💸 Total Opportunity {'Cost' if fy_opportunity > 0 else 'Saved'}:")
    print(f"   ₹{abs(fy_opportunity):,.2f}")
    if fy_opportunity > 0:
        print(f"   Note: Additional profit missed by selling early in {fy}")
    else:
        print(f"   Note: Loss avoided by selling when you did in {fy}")

print(f"\n{'=' * 80}")
print(f"📈 ALL-TIME SUMMARY ({len(result_df)} total trades)")
print(f"{'=' * 80}")

print("\n📊 Overall Buy Decision Performance:")
buy_verdict_counts_all = result_df['Buy Verdict'].value_counts()
for verdict, count in buy_verdict_counts_all.items():
    percentage = (count / len(result_df)) * 100
    print(f"   {verdict}: {count} trades ({percentage:.1f}%)")

total_actual_profit = result_df['Actualised Profit'].sum()
print(f"\n💰 Total Actualised Profit/Loss (All-Time):")
print(f"   ₹{total_actual_profit:,.2f}")

print("\n📊 Overall Sell Timing Analysis:")
sell_verdict_counts_all = result_df['Sell Verdict'].value_counts()
for verdict, count in sell_verdict_counts_all.items():
    if verdict not in ['No data', 'Error']:
        percentage = (count / len(result_df)) * 100
        print(f"   {verdict}: {count} trades ({percentage:.1f}%)")

result_df_copy = result_df.copy()
result_df_copy['Opp_numeric'] = result_df_copy['Opportunity Cost/Gain'].apply(
    lambda x: float(x.replace(',', '')) if isinstance(x, str) and x != 'N/A' else 0
)
total_opportunity = result_df_copy['Opp_numeric'].sum()

print(f"\n💸 Total Opportunity {'Cost' if total_opportunity > 0 else 'Saved'} (All-Time):")
print(f"   ₹{abs(total_opportunity):,.2f}")

print("\n" + "─" * 80)
print("📌 Key Insights:")

fy_profit_summary = result_df.groupby('Exit FY')['Actualised Profit'].sum().sort_values()
best_profit_fy = fy_profit_summary.index[-1]
worst_profit_fy = fy_profit_summary.index[0]

print(f"   ✅ Best Profit FY: {best_profit_fy} (₹{fy_profit_summary[best_profit_fy]:,.2f})")
print(f"   ⚠️  Worst Profit FY: {worst_profit_fy} (₹{fy_profit_summary[worst_profit_fy]:,.2f})")

good_buy_calls = buy_verdict_counts_all.get('Good call', 0)
good_sell_calls = sell_verdict_counts_all.get('Good call', 0)
buy_success_rate = (good_buy_calls / len(result_df)) * 100
sell_success_rate = (good_sell_calls / len(result_df)) * 100

print(f"\n   🎯 Buy Success Rate: {buy_success_rate:.1f}% (Profitable trades)")
print(f"   🎯 Sell Timing Success Rate: {sell_success_rate:.1f}% (Avoided losses)")

avg_profit = result_df['Actualised Profit'].mean()
print(f"\n   📊 Average Profit per Trade: ₹{avg_profit:,.2f}")

print("\n" + "=" * 80)
print("✓ Analysis complete! Detailed results will be saved in the next step.")
print("=" * 80)


TRADING PERFORMANCE ANALYSIS - BY FINANCIAL YEAR

────────────────────────────────────────────────────────────────────────────────
📅 FY 2025-26 (719 trades)
────────────────────────────────────────────────────────────────────────────────

📊 Buy Decision Performance:
   Good call: 451 trades (62.7%)
   Bad call: 268 trades (37.3%)

💰 Total Actualised Profit/Loss:
   ₹102,876.52
   Note: Net profit from all trades closed in FY 2025-26

📊 Sell Timing Analysis:
   Wrong early sell: 401 trades (55.8%)
   Good call: 308 trades (42.8%)

💸 Total Opportunity Cost:
   ₹1,133,149.51
   Note: Additional profit missed by selling early in FY 2025-26

📈 ALL-TIME SUMMARY (719 total trades)

📊 Overall Buy Decision Performance:
   Good call: 451 trades (62.7%)
   Bad call: 268 trades (37.3%)

💰 Total Actualised Profit/Loss (All-Time):
   ₹102,876.52

📊 Overall Sell Timing Analysis:
   Wrong early sell: 401 trades (55.8%)
   Good call: 308 trades (42.8%)

💸 Total Opportunity Cost (All-Time):
   ₹1,133,1

## Step 9: Save and Download Results

In [ ]:
# Save to CSV (keep original Actualised Profit column for calculations)
output_df = result_df[[
    'Symbol',
    'Entry Date',
    'Buy Value',
    'Buy Conditions',
    'Exit Date',
    'Sell Value',
    'Sell Conditions',
    'Quantity',
    'Actualised Profit',
    'Buy Verdict',
    'Current Price',
    'Price Change Since Sell',
    'Sell Verdict',
    'Opportunity Cost/Gain'
]].copy()

output_filename = 'analyzed_trades_complete.csv'
output_df.to_csv(output_filename, index=False)
print(f"✓ Results saved to '{output_filename}'")

# Download the file
print("\nDownloading the results file...")
files.download(output_filename)
print("\n✓ Download complete!")

In [ ]:


#possible to fix stock spilt inaccuracies?

#possible to fix stock name changes and "-" issues

###########################################################

#understand what you are doing right or wrong individually
#is it influenced by news?
#understand if there is a pattern wrt dma
#share the selling points with llm and check once as well
#market just fell and you acted to protect capital?, acted on SL rules?


In [ ]:


# TODO - FUTURE ENHANCEMENTS:
# [ ] Fix stock split inaccuracies (detect and adjust historical prices)
# [ ] Handle stock name changes and "-" issues (create mapping)
# [ ] Understand what you are doing right or wrong individually
# [ ] Check if decisions were influenced by news (integrate news API?)
# [ ] Find patterns with respect to DMA (statistical analysis)
# [ ] Share selling points with LLM for analysis
# [ ] Add context: Did market fall? Stop-loss triggered?
# [ ] Add sector/industry analysis
# [ ] Add holding period analysis
# [ ] Compare against index performance (Nifty/Sensex)


## 🎉 Analysis Complete!

### What to do next:

1. **Review Buy Decisions**: Look at "Buy Conditions" and "Buy Verdict" to learn from profitable vs loss-making entries
2. **Review Sell Timing**: Check "Sell Conditions" and "Sell Verdict" to understand if you sold at the right time
3. **Study Patterns**: Look for patterns in market conditions (DMA positions) when you made good vs bad decisions
4. **Calculate Impact**: Review "Actualised Profit" (your actual gain/loss) and "Opportunity Cost/Gain" (what you missed/saved)

### Understanding the output:

**Buy Conditions**: Shows where the price was relative to moving averages when you bought
- Positive % = Price was above DMA (expensive, trending up)
- Negative % = Price was below DMA (cheap, potential value)

**Buy Verdict**:
- "Good call" = You made profit (Actualised Profit > 0)
- "Bad call" = You took a loss (Actualised Profit < 0)

**Sell Conditions**: Shows where the price was relative to moving averages when you sold
- Positive % = Sold when price was above DMA (strength)
- Negative % = Sold when price was below DMA (weakness)

**Sell Verdict**:
- "Good call" = Price went down after you sold (avoided further loss)
- "Wrong early sell" = Price went up after you sold (missed gains)

**Opportunity Cost/Gain**:
- Positive = Money you could have made by holding longer
- Negative = Money you saved by selling when you did

---

**Pro Tip**: Keep this notebook and re-run it with updated tax P&L files to track your trading decisions over time!